<a href="https://colab.research.google.com/github/francji1/01NAEX/blob/main/code/01NAEX_Exercise_09_student_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# O1NAEX Exercise 09

In [ ]:
# cat(system('sudo apt-get install -y gmp', intern=TRUE), sep = "\n")
# cat(system('sudo apt-get install -y partitions', intern=TRUE), sep = "\n")
# cat(system('sudo apt-get install -y DoE.base', intern=TRUE), sep = "\n")

# list_of_packages <- c("tidyverse","MASS","FrF2") # "agricolae","nortest","lattice","pwr",
# missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
# missing_packages

# if(length(missing_packages)) install.packages(missing_packages)
# lapply(list_of_packages, library, character.only = TRUE)
install.packages('rsm')
install.packages('ggplot2')
install.packages('car')
install.packages('reshape2')

In [ ]:
install.packages('metR')

In [ ]:
library('rsm')
library('ggplot2')
library('reshape2')
library(dplyr)
library(car)
library('metR')

In [ ]:
knitr::opts_chunk$set(echo = FALSE,warning = FALSE, message = FALSE,fig.width=15, fig.height=10)

In [ ]:
# list_of_packages <- c("tidyverse","MASS","FrF2","dplyr","lattice","ggplot2","Hmisc")
# missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
# print(missing_packages)

# system("sudo apt-get install libgmp-dev")
# install.packages("gmp")

# system("sudo apt-get install libpartitions-dev")
# install.packages("partitions")

# system("sudo apt-get install libDoE.base-dev")
# install.packages("DoE.base")

# if(length(missing_packages)) install.packages(missing_packages)
# lapply(list_of_packages, library, character.only = TRUE)

In [ ]:
#tibble::tibble(
#  Package = names(installed.packages()[,3]),
#  Version = unname(installed.packages()[,3])
#)

##	Problem 11.8
from the chapter 11 -  D. C. Montgomery DAoE - 8. edition.

The data  were collected in	an experiment to optimize crystal growth as a function of	three variables $x_1$, $x_2$, and $x_3$. Large values of y (`Yield` in grams)	are desirable. Fit a second-order model and analyze the fitted surface. Under what set of conditions is maximum growth achieved?

In [ ]:
df118 <- read.table("https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex118.csv",header=TRUE,sep=";")
df118

Quadratic model with interactions:

In [ ]:
full_model <- lm(Yield ~ x1 + x2 + x3 + I(x1^2) + I(x2^2) + I(x3^2) + x1:x2 + x1:x3 + x2:x3, data = df118)
model_summary<-summary(full_model)
model_summary
full_model %>%
  aov() %>%
  summary()

In [ ]:
coefficients_table <- model_summary$coefficients

# Confidence intervals for the model parameters
conf_intervals <- confint(full_model)

# Calculate VIF for the model
model_vif <- vif(full_model)
model_vif <- c(NA, vif(full_model))

results <- data.frame(
  Coefficient_Estimate = round(coefficients_table[, "Estimate"],2),
  Standard_Error = round(coefficients_table[, "Std. Error"],2),
  CI_95_Low = round(conf_intervals[, 1],2),
  CI_95_High = round(conf_intervals[, 2],2),
  VIF = round(model_vif,2)
)
rownames(results) <- rownames(coefficients_table)

# Print the results
knitr::kable(results)

A lot of the terms seem to be non-significant. Therefore, the model is reduced.

In [ ]:
reduced_model <- lm(Yield ~  x2 + x3 + I(x2^2) + I(x3^2), data = df118)
red_model_summary<-summary(reduced_model)
red_model_summary
reduced_model%>%
  aov() %>%
  summary()

In [ ]:

coefficients_table <- red_model_summary$coefficients

# Confidence intervals for the model parameters
conf_intervals <- confint(reduced_model)

# Calculate VIF for the model
model_vif <- c(NA, vif(reduced_model))

results <- data.frame(
  Coefficient_Estimate = round(coefficients_table[, "Estimate"],2),
  Standard_Error = round(coefficients_table[, "Std. Error"],2),
  CI_95_Low = round(conf_intervals[, 1],2),
  CI_95_High = round(conf_intervals[, 2],2),
  VIF = round(model_vif,2)
)
rownames(results) <- rownames(coefficients_table)

# Print the results
knitr::kable(results)

Based on the F-statistic, the model seems to be significant.


Contourplot is shown below.

In [ ]:
contour(reduced_model, ~x2 + x3, image=TRUE)
coefficients <- reduced_model$coefficients

b <- c(coefficients['x2'], coefficients['x3'])
B <- matrix(0, nrow = 2, ncol = 2)
diag(B) <- c(coefficients['I(x2^2)'], coefficients['I(x3^2)'])

stationary_point <- -1/2 * solve(B) %*% b
stationary_point_natural <- stationary_point

stationary_response <- predict(reduced_model, newdata = data.frame(x2 = stationary_point_natural[1],
                                                                  x3 = stationary_point_natural[2]))

points(stationary_point_natural[1,1],stationary_point_natural[2,1],pch=19,col='red')
#include design points
#points(df118$x2, df118$x3, pch = 19, col = 'blue')
stationary_response
stationary_point_natural

Value of the yield variable predicted by the model at the stationary point: 97.68.

##	Problem 11.12
from the chapter 11 -  D. C. Montgomery DAoE - 8. edition.

Consider the three-variable central composite design. Analyze the data and draw conclusions, assuming that we wish to maximize `Conversion` ($y_1$) with			`Activity` ($y_2$) between 55 and 60	achieved?

In [ ]:
df1112 <- read.table("https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex1112.csv",header=TRUE,sep=";")
df1112

Quadratic model for conversion:

In [ ]:
conversion_model <- lm(Conversion ~Time +Temperature+ Catalyst+Time:Temperature+Time:Catalyst+Temperature:Catalyst+I(Time^2)+I(Temperature^2)+I(Catalyst^2), data = df1112)
conversion_summary<-summary(conversion_model)
conversion_summary
conversion_model%>%
  aov() %>%
  summary()

The p-value suggest that the model is significant.

Model coefficient summary table:

In [ ]:
coefficients_table <- conversion_summary$coefficients
conf_intervals <- confint(conversion_model)
model_vif <- vif(conversion_model)
model_vif <- c(NA, vif(conversion_model))
results <- data.frame(
  Coefficient_Estimate = round(coefficients_table[, "Estimate"],2),
  Standard_Error = round(coefficients_table[, "Std. Error"],2),
  CI_95_Low = round(conf_intervals[, 1],2),
  CI_95_High = round(conf_intervals[, 2],2),
  VIF = round(model_vif,2)
)
rownames(results) <- rownames(coefficients_table)
knitr::kable(results)

Model for activity

In [ ]:
Activity_model <- lm(Activity ~Time +Temperature+ Catalyst+Time:Temperature+Time:Catalyst+Temperature:Catalyst+I(Time^2)+I(Temperature^2)+I(Catalyst^2), data = df1112)
Activity_summary<-summary(Activity_model)
Activity_summary
Activity_model%>%
  aov() %>%
  summary()

P-value implies that the model should be significant. However most of the parameters seem insignificant.

Summary table of this model:

In [ ]:
coefficients_table <- Activity_summary$coefficients
conf_intervals <- confint(Activity_model)
model_vif <- vif(Activity_model)
model_vif <- c(NA, vif(Activity_model))
results <- data.frame(
  Coefficient_Estimate = round(coefficients_table[, "Estimate"],2),
  Standard_Error = round(coefficients_table[, "Std. Error"],2),
  CI_95_Low = round(conf_intervals[, 1],2),
  CI_95_High = round(conf_intervals[, 2],2),
  VIF = round(model_vif,2)
)
rownames(results) <- rownames(coefficients_table)
knitr::kable(results)

Reducing the activity model:

In [ ]:
Activity_model_reduced <- lm(Activity ~Time + Catalyst+I(Time^2), data = df1112)
Activity_reduced_summary<-summary(Activity_model_reduced)
Activity_reduced_summary
Activity_model_reduced%>%
  aov() %>%
  summary()

In [ ]:
coefficients_table <- Activity_reduced_summary$coefficients
conf_intervals <- confint(Activity_model_reduced)
model_vif <- vif(Activity_model_reduced)
model_vif <- c(NA, vif(Activity_model_reduced))
results <- data.frame(
  Coefficient_Estimate = round(coefficients_table[, "Estimate"],2),
  Standard_Error = round(coefficients_table[, "Std. Error"],2),
  CI_95_Low = round(conf_intervals[, 1],2),
  CI_95_High = round(conf_intervals[, 2],2),
  VIF = round(model_vif,2)
)
rownames(results) <- rownames(coefficients_table)
knitr::kable(results)

All the variables now seem to be significant as well as the model.

Following are contour plots for the two final models, first one for conversion, second for activity.

In [ ]:
contour(conversion_model,~Time + Catalyst,image=TRUE)
contour(Activity_model_reduced,~Time + Catalyst,image=TRUE)

Overlay plots:

In [ ]:
new_data <- expand.grid(Time = unique(df1112$Time), Catalyst = unique(df1112$Catalyst), Temperature = -1)

new_data$Conversion <- predict(conversion_model, newdata = new_data)
new_data$Activity <- predict(Activity_model_reduced, newdata = new_data)


new_data$response = "Conversion"
new_data2 <- new_data
new_data2$response = "Activity"
new_data2$Conversion <- new_data$Activity
combined_data <- rbind(new_data, new_data2)

library(ggplot2)



g <- ggplot(combined_data, aes(Time, Catalyst)) +
  geom_contour(data = subset(combined_data, response == "Conversion"), aes(z = Conversion), color = "blue",binwidth = 10) +
  geom_contour(data = subset(combined_data, response == "Activity"), aes(z = Activity), color = "red",binwidth = 10) +
  geom_text_contour(data = subset(combined_data, response == "Conversion"), aes(z = Conversion), color = "blue",binwidth=4,check_overlap = F) +
  geom_text_contour(data = subset(combined_data, response == "Activity"), aes(z = Activity),color = "red",binwidth=3,check_overlap = F)

print(g)